In [ ]:
##### Code forked from WalkerDB: https://github.com/walkerdb/supreme_court_transcripts/blob/master/update.py
import json
import requests
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=10, period=10)  # no more than 1 call per second
def get_http_json(url):
    print(f"Getting {url}")
    response = requests.get(url)
    parsed = response.json()
    return parsed


def get_case(term, docket):
    """Get the info of the case and fetch all
    transcripts that the info links to"""
    url = f"https://api.oyez.org/cases/{term}/{docket}"
    docket_data = get_http_json(url)

    return docket_data


def write_case(term, docket, docket_data):
    """
    Writes term-docket.json file with docket_data
    For each transcript, writes the term-docket-t##.json file
    """
    with open(f"case_briefs_up_to_2025/{term}.{docket}.json", "a") as docket_file:
        json.dump(docket_data, docket_file, indent=2)


In [ ]:
import os

transcripts_path = "./transcripts_up_to_2024"
# for each transcript file, pull its corresponding brief from Oyez
for filename in os.listdir(transcripts_path):
    file_path = os.path.join(transcripts_path, filename)
    if os.path.isfile(file_path):
        term = filename[:4]
        docket = filename[5:-9]
        docket_data = get_case(term, docket)

        write_case(term, docket, docket_data)
        print(f"{filename}")